## Initial imports and setup

In [ ]:
from astropy.coordinates import SkyCoord
from astropy import coordinates
from astropy.io import fits, votable
from ipyaladin import Aladin
from urllib.parse import urlparse, parse_qs, quote
from astroquery.cds import cds
from astroquery.simbad import Simbad
from regions import CircleSkyRegion
from astropy.visualization import (MinMaxInterval, PercentileInterval, SqrtStretch, AsinhStretch,
                                   ImageNormalize)
import matplotlib.pyplot as plt
import seaborn as sns
import pyvo as vo

import warnings
warnings.simplefilter("ignore")

%matplotlib inline

## Retrieve parameters from query string

In [150]:
# Parameters:

In [ ]:
%%javascript
IPython.notebook.kernel.execute("notebook_url = '" + window.location + "'");


In [ ]:
query = urlparse(notebook_url).query
#object_name = params['object']
params = parse_qs(query)
object_name = params.get('object')
if object_name is None:
    print('object parameter has not been found in URL!')
else:
    object_name = object_name[0]
    print(f'Let\'s study {object_name}')

## Retrieve object coordinates

In [ ]:
sc = SkyCoord.from_name(object_name)
sc

## Retrieve basic information from SIMBAD

In [ ]:
Simbad.reset_votable_fields()
Simbad.add_votable_fields('pmra', 'pmdec', 'plx', 'flux(B)', 'flux(V)')
Simbad.query_object(object_name)
#Simbad.list_votable_fields()

## List of identifiers

In [ ]:
Simbad.query_objectids(object_name)

## Retrieve list of published papers citing this object

In [ ]:
query = """SELECT BIBCode, Journal, Title, "year", Volume, Page || '-' || Last_Page AS "Pages", DOI
              FROM ref JOIN has_ref ON oidbibref = oidbib JOIN ident ON has_ref.oidref = ident.oidref
              WHERE id = '{}'
              ORDER BY "year" DESC""".format(object_name)
query = query.replace('\n', '')

tap_service = vo.dal.TAPService('https://simbad.u-strasbg.fr/simbad/sim-tap')
tap_results = tap_service.search(query)
tap_results

## Make FITS cutouts centered on the region (calling hips2fits)

In [ ]:
hips_list = ['DSS2/red', '2MASS/K', 'PanSTARRS/DR1/g', 'AllWISE/W4']
fig, axs = plt.subplots(1, len(hips_list), figsize=(4 * len(hips_list), 3), facecolor='w', edgecolor='k')
fig.subplots_adjust(hspace = .2, wspace=.001)

axs = axs.ravel()

i = 0
im_width = im_height = 150
fov = 0.2
for hips in hips_list:
    axs[i].set_axis_off()

    axs[i].set_title('{} - {}'.format(object_name, hips))

    url = 'http://alasky.u-strasbg.fr/hips-image-services/hips2fits?hips={}&width={}&height={}&fov={}&projection=TAN&coordsys=icrs&ra={}&dec={}'.format(quote(hips), im_width, im_height, fov, sc.icrs.ra.deg, sc.icrs.dec.deg)
    
    hdu = fits.open(url)
        
    im = hdu[0].data
    norm = ImageNormalize(im, interval=MinMaxInterval(),
                  stretch=AsinhStretch())
    axs[i].imshow(im, cmap='magma', norm=norm, origin='lower')
        
    i += 1

## Display object in ipyaladin

In [ ]:
al = Aladin(target=object_name, fov=0.4)
al

## Query the MocServer to find catalogues available in the vicinity of the region

In [ ]:
r = cds.query_region(CircleSkyRegion(sc, coordinates.Angle(0.1, unit='deg')), fields=('ID', 'dataproduct_type','cs_service_url'))
catalogs = r[r['dataproduct_type']=='catalog']
catalogs

## Retrieve photometric points

In [ ]:
photometry_points_url = 'http://vizier.u-strasbg.fr/viz-bin/sed?-c={}&-c.rs=3'.format(quote(object_name))
phot_result = votable.parse_single_table(photometry_points_url).to_table()

In [ ]:
plt.figure(figsize=(12, 8))
sns.set()

c = 3e8
#plt.scatter(x=phot_result['sed_freq'], y=phot_result['sed_flux'])
splot = sns.scatterplot(x=c/phot_result['sed_freq'], y=phot_result['sed_freq']*phot_result['sed_flux'])
                    
splot.set(xscale="log")
splot.set(yscale="log")
splot.set(xlabel='λ', ylabel='nu.F_nu')


## Are there associated VizieR data available?

In [ ]:
query = "SELECT TOP 500 * FROM obscore WHERE 1=CONTAINS(POINT('ICRS', s_ra, s_dec),CIRCLE('ICRS',{}, {}, 0.5))".format(sc.icrs.ra.deg, sc.icrs.dec.deg)

tap_service = vo.dal.TAPService('http://cdsarc.u-strasbg.fr/saadavizier.tap/tap')
tap_results = tap_service.search(query)
tap_results.to_table()